In [8]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [9]:
btc = pd.read_csv("../data/raw/btc_prices.csv", parse_dates=["date"])
nifty = pd.read_csv("../data/raw/nifty_prices.csv", parse_dates=["date"])

btc["date"] = btc["date"].dt.tz_localize(None)
nifty["date"] = nifty["date"].dt.tz_localize(None)

btc.head(), nifty.head()

(        date        open        high         low       close    volume  \
 0 2015-01-02  314.079010  315.838989  313.565002  315.032013   7860650   
 1 2015-01-03  314.846008  315.149994  281.082001  281.082001  33054400   
 2 2015-01-04  281.145996  287.230011  257.612000  264.195007  55629100   
 3 2015-01-05  265.084015  278.341003  265.084015  274.473999  43962800   
 4 2015-01-06  274.610992  287.553009  272.696014  286.188995  23245700   
 
      return  
 0  0.002492  
 1 -0.107767  
 2 -0.060079  
 3  0.038907  
 4  0.042682  ,
         date         open         high          low        close  volume  \
 0 2010-01-05  5277.149902  5288.350098  5242.399902  5277.899902       0   
 1 2010-01-06  5278.149902  5310.850098  5260.049805  5281.799805       0   
 2 2010-01-07  5281.799805  5302.549805  5244.750000  5263.100098       0   
 3 2010-01-08  5264.250000  5276.750000  5234.700195  5244.750000       0   
 4 2010-01-11  5263.799805  5287.200195  5227.799805  5249.399902       

In [10]:
def fetch_fear_greed():
    url = "https://api.alternative.me/fng/?limit=0&format=json"
    r = requests.get(url).json()["data"]

    df = pd.DataFrame(r)
    df["timestamp"] = df["timestamp"].astype(float)
    df["date"] = pd.to_datetime(df["timestamp"], unit="s")
    df["fear_greed"] = df["value"].astype(float)

    return df[["date", "fear_greed"]].sort_values("date")

fear_greed = fetch_fear_greed()
fear_greed.head()

,date,fear_greed
2897,2018-02-01,30.0
2896,2018-02-02,15.0
2895,2018-02-03,40.0
2894,2018-02-04,24.0
2893,2018-02-05,11.0


In [11]:
gt_btc = pd.read_csv("../data/raw/btc_google_trends.csv", parse_dates=["date"])
gt_nifty = pd.read_csv("../data/raw/nifty_google_trends.csv", parse_dates=["date"])
gt_btc.head()
gt_nifty.head()

,date,google_trend
0,2010-01-01,22
1,2010-02-01,25
2,2010-03-01,22
3,2010-04-01,21
4,2010-05-01,28


In [12]:
def compute_vol_sentiment(df, window=14):
    df = df.sort_values("date").copy()
    df["return"] = df["close"].pct_change()
    df["vol"] = df["return"].rolling(window).std()

    # Normalize
    scaler = MinMaxScaler()
    df["vol_sent"] = scaler.fit_transform(df[["vol"]])

    return df[["date", "vol_sent"]]

btc_vol_sent = compute_vol_sentiment(btc)
nifty_vol_sent = compute_vol_sentiment(nifty)

In [13]:
btc_long = btc[["date", "close"]].merge(
    fear_greed, on="date", how="left"
).merge(
    gt_btc[["date", "google_trend"]], on="date", how="left"
).merge(
    btc_vol_sent, on="date", how="left"
)
btc_long[["date", "google_trend"]].dropna().head()
btc_long[["date", "google_trend"]].dropna().tail()
btc_long.head()

,date,close,fear_greed,google_trend,vol_sent
0,2015-01-02,315.032013,NaN,NaN,NaN
1,2015-01-03,281.082001,NaN,NaN,NaN
2,2015-01-04,264.195007,NaN,NaN,NaN
3,2015-01-05,274.473999,NaN,NaN,NaN
4,2015-01-06,286.188995,NaN,NaN,NaN


In [14]:
nifty_long = nifty[["date", "close"]].merge(
    gt_nifty[["date", "google_trend"]], on="date", how="left"
).merge(
    nifty_vol_sent, on="date", how="left"
)

nifty_long.head()

,date,close,google_trend,vol_sent
0,2010-01-05,5277.899902,NaN,NaN
1,2010-01-06,5281.799805,NaN,NaN
2,2010-01-07,5263.100098,NaN,NaN
3,2010-01-08,5244.750000,NaN,NaN
4,2010-01-11,5249.399902,NaN,NaN
